In [2]:
# copy example repository


Cloning into 'test_repo'...
remote: Not Found
fatal: repository 'https://github.com/gryazevae/' not found


In [6]:
import matplotlib.pyplot as plt
import subprocess
from datetime import datetime

def get_commit_history():
    git_log = subprocess.Popen(['git', 'log', '--all', '--format=%ct'], stdout=subprocess.PIPE)
    output, _ = git_log.communicate()
    commit_dates = output.decode().split('\n')[:-1]
    return sorted([datetime.fromtimestamp(int(date)) for date in commit_dates])

def count_commits_per_month(commit_dates):
    first_date = commit_dates[0].replace(day=1, hour=0, minute=0, second=0, microsecond=0)                # Устанавливаем начальную дату на первое число текущего месяца
    last_date = commit_dates[-1].replace(day=1, month= (commit_dates[-1].month+1) % 12 + ((commit_dates[-1].month+1) // 12), year = commit_dates[-1].year + ((commit_dates[-1].month+1) // 12))       
    start_date = first_date               # Устанавливаем начальную дату на первое число текущего месяца
    end_date = start_date.replace(day=1, month= (start_date.month+1) % 12 + ((start_date.month+1) // 12), year = start_date.year + ((start_date.month+1) // 12))                                      # Устанавливаем конечную дату на первое число следующего месяца
    commit_counts = []
    while end_date <= last_date:
        count = 0
        for date in commit_dates:
            if (date <= end_date) and (date >= start_date) :
                count += 1
        commit_counts.append(count)
        start_date = end_date
        #print(end_date)
        #print((end_date.month+1) % 12  + ((start_date.month+1) // 12))
        end_date = end_date.replace(day=1, month= (end_date.month+1) % 12 + ((start_date.month+1) // 12), year = end_date.year + ((start_date.month+1) // 12))  
    return commit_counts

def plot_commit_frequency(commit_dates, commit_counts):
    plt.bar(commit_dates, commit_counts, width=timedelta(days=30))
    plt.xlabel('Месяц')
    plt.ylabel('Частота коммитов')
    plt.title('Сумма коммитов каждый месяц во всех ветках')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [7]:
!cd ./test_repo

In [8]:
commit_dates = get_commit_history()
commit_counts = count_commits_per_month(commit_dates)
plt.plot(commit_counts, '-o')
plt.xlabel('Месяц')
plt.ylabel('Частота коммитов')
plt.title('Сумма коммитов каждый месяц во всех ветках')
#plot_commit_frequency(commit_dates, commit_counts)

IndexError: list index out of range